## Llama 3.1 Function Calling 예제

In [1]:
import json
import re
import yfinance as yf
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer
from datetime import datetime

INFO 04-21 08:59:39 [__init__.py:239] Automatically detected platform cuda.


In [2]:
# 모델 및 토크나이저 초기화
MODEL_ID = "meta-llama/Llama-3.1-8B-Instruct"
model = LLM(MODEL_ID, tensor_parallel_size=1)
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

INFO 04-21 08:59:58 [config.py:600] This model supports multiple tasks: {'reward', 'score', 'classify', 'generate', 'embed'}. Defaulting to 'generate'.
INFO 04-21 08:59:58 [config.py:1780] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 04-21 09:00:00 [core.py:61] Initializing a V1 LLM engine (v0.8.3) with config: model='meta-llama/Llama-3.1-8B-Instruct', speculative_config=None, tokenizer='meta-llama/Llama-3.1-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=131072, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar', reasoning_backend=None), observability_config=ObservabilityConfig(show_hidden_metrics=Fal

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 04-21 09:01:26 [loader.py:447] Loading weights took 81.50 seconds
INFO 04-21 09:01:27 [gpu_model_runner.py:1273] Model loading took 14.9889 GiB and 83.196120 seconds
INFO 04-21 09:01:35 [backends.py:416] Using cache directory: /giant-data/user/1111332/.cache/vllm/torch_compile_cache/839addc9a7/rank_0_0 for vLLM's torch.compile
INFO 04-21 09:01:35 [backends.py:426] Dynamo bytecode transform time: 8.13 s
INFO 04-21 09:01:36 [backends.py:115] Directly load the compiled graph for shape None from the cache
INFO 04-21 09:01:45 [monitor.py:33] torch.compile takes 8.13 s in total
INFO 04-21 09:01:47 [kv_cache_utils.py:578] GPU KV cache size: 417,568 tokens
INFO 04-21 09:01:47 [kv_cache_utils.py:581] Maximum concurrency for 131,072 tokens per request: 3.19x
INFO 04-21 09:02:23 [gpu_model_runner.py:1608] Graph capturing finished in 36 secs, took 1.61 GiB
INFO 04-21 09:02:23 [core.py:162] init engine (profile, create kv cache, warmup model) took 56.21 seconds


In [3]:
# 샘플링 파라미터 설정 - llama3.1 에 맞게 stop token 변경
sampling_params_func_call = SamplingParams(
    max_tokens=256, temperature=0.0, stop=["<|im_end|>"], skip_special_tokens=False
)
sampling_params_text = SamplingParams(
    max_tokens=512, temperature=0.1, top_p=0.95, stop=["<|im_end|>"], skip_special_tokens=False
)

In [4]:
# KOSPI 주식 정보
KOSPI_TICKER_MAP = {
    "SK텔레콤": "017670.KS", "삼성전자": "005930.KS", "SK하이닉스": "000660.KS",
    "현대차": "005380.KS", "기아": "000270.KS", "LG에너지솔루션": "373220.KS",
    "NAVER": "035420.KS", "카카오": "035720.KS",
}

In [5]:
# 도구(함수) 정의
TOOLS = [{
    "type": "function",
    "function": {
        "name": "get_kospi_stock_info",
        "description": "특정 KOSPI 주식의 현재 가격 및 기본 정보를 가져옵니다.",
        "parameters": {
            "type": "object",
            "properties": {
                "stock_name_or_code": {
                    "type": "string",
                    "description": "주식 이름(예: 'SK텔레콤') 또는 종목 코드(예: '017670')"
                }
            },
            "required": ["stock_name_or_code"]
        }
    }
}]

In [6]:
# 주가 조회
def get_kospi_stock_info(stock_name_or_code: str) -> str:

    name_or_code = stock_name_or_code.strip()
    ticker_symbol = None

    if re.fullmatch(r'\d{6}', name_or_code):
        ticker_symbol = name_or_code + ".KS"
    elif name_or_code in KOSPI_TICKER_MAP:
        ticker_symbol = KOSPI_TICKER_MAP[name_or_code]
    else:
        ticker_symbol = name_or_code + ".KS"

    stock = yf.Ticker(ticker_symbol)
    stock_info = stock.info

    current_price = stock_info.get('currentPrice')
    previous_close = stock_info.get('previousClose')

    price_to_use = current_price if current_price is not None else previous_close
    price_display = round(price_to_use, 2) if price_to_use is not None else "정보 없음"
    previous_close_display = round(previous_close, 2) if previous_close is not None else "정보 없음"

    result = {
        "ticker": ticker_symbol,
        "stock_name": stock_info.get('shortName', name_or_code),
        "current_price": price_display,
        "previous_close": previous_close_display,
        "currency": stock_info.get('currency', 'KRW')
    }

    return json.dumps(result, ensure_ascii=False)

In [7]:
# 함수 호출 파싱
# JSON 유효성 검사
def parse_tool_calls(output_text: str):

    cleaned_output = output_text.strip()
    json.loads(cleaned_output)

    return {
        "role": "assistant",
        "content": cleaned_output
    }

In [8]:
# IPython 메시지 생성 함수
def create_ipython_message(tool_result_dict: dict):
    return {
        "role": "ipython",
        "content": {
            "output": tool_result_dict
        }
    }

In [9]:
# 메인 쿼리 처리 함수
def query_kospi_info(query: str):

    current_date = datetime.now().strftime('%Y-%m-%d')
    system_message = f"When you receive a tool call response, use the output to format an answer to the orginal user question. \n You are a helpful assistant with tool calling capabilities. Current Date: {current_date}"

    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": query}
    ]
    print(f"\n### 1단계 초기 메시지 작성:\n{messages}")

    prompt = tokenizer.apply_chat_template(messages, tools=TOOLS, add_generation_prompt=True, tokenize=False)
    cleaned_prompt = "\n".join(
        line for line in prompt.splitlines() if "Environment: ipython" not in line
    )
    print(f"\n### 2단계 함수 선택/응답 생성을 위한 프롬프트 구성:\n{cleaned_prompt}")

    outputs = model.generate([cleaned_prompt], sampling_params=sampling_params_func_call)
    func_call_response_text = outputs[0].outputs[0].text
    print(f"\n### 3단계 함수 호출/초기 응답을 위한 LLM 응답:\n{func_call_response_text}")

    assistant_msg = parse_tool_calls(func_call_response_text)
    messages.append(assistant_msg)
    print(f"\n### 4단계 어시스턴트 함수 호출 메시지 추가 후:\n{messages}")

#    tool_call_content = assistant_msg['content']
    tool_call_content = func_call_response_text
    print(f"$$$ check: {tool_call_content}")
    python_tag = "<|python_tag|>"
    if tool_call_content.startswith(python_tag):
        tool_call_content = tool_call_content[len(python_tag):].strip()

    tool_data = json.loads(tool_call_content)
    tool_params = tool_data.get("parameters")

    if tool_params is None or "stock_name_or_code" not in tool_params:
         raise ValueError("LLM response did not contain valid 'parameters' or 'stock_name_or_code'")

    stock_name_or_code = tool_params["stock_name_or_code"]
    function_result_json_str = get_kospi_stock_info(stock_name_or_code)

    function_result_dict = json.loads(function_result_json_str)
    print(f"\n### 5단계 함수 실행 결과:\n{function_result_dict}")

    ipython_msg = create_ipython_message(function_result_dict)
    messages.append(ipython_msg)
    print(f"\n### 5.1단계 Tool 결과('ipython' role) 메시지 추가 후:\n{messages}")

    final_prompt = tokenizer.apply_chat_template(messages, tools=TOOLS, add_generation_prompt=True, tokenize=False)

    print(f"\n### 6단계 최종 응답 생성을 위한 프롬프트:\n{final_prompt}")
    
    outputs = model.generate([final_prompt], sampling_params=sampling_params_text)
    final_response = outputs[0].outputs[0].text

    return final_response

In [10]:
if __name__ == "__main__":
    queries = [
        "Tell me the stock price of SK텔레콤",
#        "현대차 주가 알려줘"
    ]

    for query in queries:
        print(f"\n========================================")
        print(f" 질문: {query}")
        print(f"========================================")
        response = query_kospi_info(query)
        print(f"\n 답변: {response}")


 질문: Tell me the stock price of SK텔레콤

### 1단계 초기 메시지 작성:
[{'role': 'system', 'content': 'When you receive a tool call response, use the output to format an answer to the orginal user question. \n You are a helpful assistant with tool calling capabilities. Current Date: 2025-04-21'}, {'role': 'user', 'content': 'Tell me the stock price of SK텔레콤'}]

### 2단계 함수 선택/응답 생성을 위한 프롬프트 구성:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

When you receive a tool call response, use the output to format an answer to the orginal user question. 
 You are a helpful assistant with tool calling capabilities. Current Date: 2025-04-21<|eot_id|><|start_header_id|>user<|end_header_id|>

Given the following functions, please respond with a JSON for a function call with its proper arguments that best answers the given prompt.

Respond in the format {"name": function name, "parameters": dictionary of argument name and its value}.Do no

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  2.68it/s, est. speed input: 736.34 toks/s, output: 75.24 toks/s]



### 3단계 함수 호출/초기 응답을 위한 LLM 응답:
{"name": "get_kospi_stock_info", "parameters": {"stock_name_or_code": "SK텔레콤"}}

### 4단계 어시스턴트 함수 호출 메시지 추가 후:
[{'role': 'system', 'content': 'When you receive a tool call response, use the output to format an answer to the orginal user question. \n You are a helpful assistant with tool calling capabilities. Current Date: 2025-04-21'}, {'role': 'user', 'content': 'Tell me the stock price of SK텔레콤'}, {'role': 'assistant', 'content': '{"name": "get_kospi_stock_info", "parameters": {"stock_name_or_code": "SK텔레콤"}}'}]
$$$ check: {"name": "get_kospi_stock_info", "parameters": {"stock_name_or_code": "SK텔레콤"}}

### 5단계 함수 실행 결과:
{'ticker': '017670.KS', 'stock_name': 'SKTelecom', 'current_price': 57700.0, 'previous_close': 57900.0, 'currency': 'KRW'}

### 5.1단계 Tool 결과('ipython' role) 메시지 추가 후:
[{'role': 'system', 'content': 'When you receive a tool call response, use the output to format an answer to the orginal user question. \n You are a helpful assistant wi

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  3.53it/s, est. speed input: 1294.16 toks/s, output: 63.81 toks/s]


 답변: The current price of SK텔레콤 is 57,700 KRW.
